In [ ]:
from data_resolvers.mongo_db_connection import MongoDBConnection
from bson.objectid import ObjectId
from data_resolvers.ciselniky_data_resolver import CiselnikyDataResolver
from data_resolvers.ciselnikyDetail_data_resolver import CiselnikyDetailDataResolver
from data_resolvers.nezrovnalost_data_resolver import NezrovnalostDataResolver
from data_resolvers.nezrovnalostDetail_data_resolver import NezrovnalostDetailDataResolver

In [ ]:
connection = MongoDBConnection('./appsettings.json')
client = connection.client

In [ ]:
client.list_database_names()
itms_db_name = 'itmsDB'
itmsDB = client.get_database(itms_db_name)
ciselniky_col_name = 'ciselniky'
ciselnikyDetail_col_name = 'ciselnikyDetail'
nezrovnalost_col_name = 'nezrovnalosti'
nezrovnalostDetail_col_name = 'nezrovnalostiDetail'

In [ ]:
# ciselniky_data_res = CiselnikyDataResolver(connection, itms_db_name, 'ciselniky')
# ciselniky_data_res.fetch_remote_data()

# ciselnikyDetail_data_res = CiselnikyDetailDataResolver(
#     connection, 
#     itms_db_name,
#     ciselniky_col_name,
#     ciselnikyDetail_col_name)
# await ciselnikyDetail_data_res.fetch_remote_data_async()

# nezrovnalost_data_res = NezrovnalostDataResolver(connection, itms_db_name, nezrovnalost_col_name)
# nezrovnalost_data_res.fetch_remote_data()

nezrovnalostDetail_data_res = NezrovnalostDetailDataResolver(
    connection, 
    itms_db_name,
    nezrovnalost_col_name,
    nezrovnalostDetail_col_name)
remote_data = await nezrovnalostDetail_data_res.fetch_remote_data_async()

In [ ]:
def get_all_local_data():
    local_ciselnikDetail = itmsDB.get_collection(ciselnikyDetail_col_name).aggregate([
        {
            '$addFields': {
                'origin': '$$CURRENT', 
                'dict_key': {
                    '$concat': [
                        {
                            '$toString': '$ciselnikKod'
                        }, '_', {
                            '$toString': '$id'
                        }
                    ]
                }
            }
        }, {
            '$project': {
                'origin._id': 0
            }
        }, {
            '$project': {
                '_id': 1, 
                'origin': 1, 
                'dict_key': 1
            }
        }
    ])
    return {item['dict_key']:item for item in local_ciselnikDetail}

In [ ]:
nezrovnaost_url_template = 'https://opendata.itms2014.sk/v2/nezrovnalost?minId={minId}'

In [ ]:
all_ids = itmsDB.get_collection(nezrovnalost_col_name).distinct("id")